<a href="https://colab.research.google.com/github/Warren-Mbekezeli-Mnisi/Logistics_Regression/blob/main/SENDY_LOGISTICS_MODELS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SENDY LOGISTICS CHALLENGE**

## **DATA PREPROCESSING**

## Import Libraries 

In [1]:
#import all the important libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Import Dataset

### Import the Training DataSet

In [2]:
#import training dataset and observe the results
train_set = pd.read_csv('Train.csv')

y_train = train_set.iloc[:, -1].values

train_set.drop(['User Id', 'Vehicle Type','Order No', 'Arrival at Destination - Weekday (Mo = 1)',
             'Arrival at Destination - Time', 'Arrival at Destination - Day of Month', 'Time from Pickup to Arrival' ],
             axis=1, inplace=True)

test_set = pd.read_csv('Test.csv')
test_set.drop(['User Id', 'Vehicle Type','Order No'  ],
             axis=1, inplace=True)

rider_info = pd.read_csv('Riders.csv')

print(len(train_set.columns))
print(train_set.columns)

print(len(test_set.columns))
print(test_set.columns)

for column in train_set.columns:
  if column not in test_set.columns:
    print(column)

22
Index(['Platform Type', 'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Placement - Time',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Confirmation - Time', 'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
       'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 'Pickup - Time',
       'Distance (KM)', 'Temperature', 'Precipitation in millimeters',
       'Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long',
       'Rider Id'],
      dtype='object')
22
Index(['Platform Type', 'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Placement - Time',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Confirmation - Time', 'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
       'Pickup - Day of Month'

## Preproccessing the Datasets

In [3]:
def convert_to_24hrs(column):
    column = pd.to_datetime(column, format='%I:%M:%S %p').dt.strftime("%H:%M:%S")
    return column

In [4]:
train_set['Confirmation - Time'] = pd.to_timedelta(convert_to_24hrs(train_set['Confirmation - Time'] ))
train_set['Placement - Time'] = pd.to_timedelta(convert_to_24hrs(train_set['Placement - Time'] ))
train_set['Arrival at Pickup - Time'] =  pd.to_timedelta(convert_to_24hrs(train_set['Arrival at Pickup - Time'] ))
train_set['Pickup - Time'] =  pd.to_timedelta(convert_to_24hrs(train_set['Pickup - Time'] ))

test_set['Confirmation - Time'] = pd.to_timedelta(convert_to_24hrs(test_set['Confirmation - Time'] ))
test_set['Placement - Time'] = pd.to_timedelta(convert_to_24hrs(test_set['Placement - Time'] ))
test_set['Arrival at Pickup - Time'] =  pd.to_timedelta(convert_to_24hrs(test_set['Arrival at Pickup - Time'] ))
test_set['Pickup - Time'] =  pd.to_timedelta(convert_to_24hrs(test_set['Pickup - Time'] ))

In [5]:
train_set['Time from Confirmation to placement'] = train_set['Confirmation - Time']- train_set['Placement - Time']
train_set['Waiting Time'] = train_set['Pickup - Time']- train_set['Arrival at Pickup - Time']

test_set['Time from Confirmation to placement'] = test_set['Confirmation - Time']- test_set['Placement - Time']
test_set['Waiting Time'] = test_set['Pickup - Time']- test_set['Arrival at Pickup - Time']

In [6]:
train_set['Time from Confirmation to placement'] = train_set['Time from Confirmation to placement'] / np.timedelta64(1, 's')
train_set['Waiting Time'] = train_set['Waiting Time'] / np.timedelta64(1, 's')

test_set['Time from Confirmation to placement'] = test_set['Time from Confirmation to placement'] / np.timedelta64(1, 's')
test_set['Waiting Time'] = test_set['Waiting Time'] / np.timedelta64(1, 's')

In [7]:
train_set['Temperature'] = train_set['Temperature'].fillna(train_set['Temperature'].mean())

test_set['Temperature'] = test_set['Temperature'].fillna(test_set['Temperature'].mean())

In [8]:
train_set['Precipitation in millimeters'] = train_set['Precipitation in millimeters'].fillna(0)

test_set['Precipitation in millimeters'] = test_set['Precipitation in millimeters'].fillna(0)

In [9]:
train_set = pd.merge(train_set, rider_info, on='Rider Id', how= 'left')

test_set = pd.merge(test_set, rider_info, on='Rider Id', how= 'left')

In [10]:
train_set.drop(['Rider Id','Confirmation - Time','Placement - Time' ,'Pickup - Time','Arrival at Pickup - Time'],
             axis=1, inplace=True)

test_set.drop(['Rider Id','Confirmation - Time','Placement - Time' ,'Pickup - Time','Arrival at Pickup - Time'],
             axis=1, inplace=True)

In [11]:
train_set.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),...,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Confirmation to placement,Waiting Time,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,3,Business,9,5,9,5,9,5,9,5,...,-1.317755,36.830370,-1.300406,36.829741,264.0,1363.0,1637,1309,13.8,549
1,3,Personal,12,5,12,5,12,5,12,5,...,-1.351453,36.899315,-1.295004,36.814358,425.0,227.0,396,339,13.6,69
2,3,Business,30,2,30,2,30,2,30,2,...,-1.308284,36.843419,-1.300921,36.828195,199.0,209.0,1023,242,12.5,114
3,3,Business,15,5,15,5,15,5,15,5,...,-1.281301,36.832396,-1.257147,36.795063,31.0,310.0,886,283,14.5,113
4,1,Personal,13,1,13,1,13,1,13,1,...,-1.266597,36.792118,-1.295041,36.809817,60.0,90.0,2311,872,14.1,533


In [12]:
test_set.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),...,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Confirmation to placement,Waiting Time,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,3,Business,27,3,27,3,27,3,27,3,...,-1.333275,36.870815,-1.305249,36.822390,19.0,823.0,439,1511,13.3,171
1,3,Business,17,5,17,5,17,5,17,5,...,-1.272639,36.794723,-1.277007,36.823907,102.0,310.0,488,273,14.4,45
2,3,Business,27,4,27,4,27,4,27,4,...,-1.290894,36.822971,-1.276574,36.851365,1011.0,1474.0,833,460,13.1,67
3,3,Business,17,1,17,1,17,1,17,1,...,-1.290503,36.809646,-1.303382,36.790658,112.0,851.0,487,560,13.7,44
4,3,Business,11,2,11,2,11,2,11,2,...,-1.281081,36.814423,-1.266467,36.792161,257.0,525.0,4761,1120,14.1,1010


In [13]:
X_train = train_set.iloc[:, :].values
X_test = test_set.iloc[:, :].values

In [14]:
y_train = y_train.reshape(len(y_train), 1)

In [15]:
print(X_train[0])
print(X_test[0])

[3 'Business' 9 5 9 5 9 5 9 5 4 20.4 0.0 -1.3177546999999998 36.8303703
 -1.3004062 36.829741 264.0 1363.0 1637 1309 13.8 549]
[3 'Business' 27 3 27 3 27 3 27 3 8 23.246119694548046 0.0 -1.3332749
 36.870814700000004 -1.3052491000000002 36.8223899 19.0 823.0 439 1511
 13.3 171]


In [16]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(len(X_train[0]))

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(drop='first'), [0,1])], remainder = 'passthrough')
X_train = np.array(ct.fit_transform(X_train))

#ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(drop='first'), [0,1])], remainder = 'passthrough')
X_test = np.array(ct.transform(X_test))

23


## Feature Scaling

In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 4:] = sc.fit_transform(X_train[:, 4:])

In [42]:
X_test[:, 4:] = sc.transform(X_test[:, 4:])

In [43]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)

## **MODEL #1**

## **MODEL #2**

## **MODEL #3**